# How to run a semantic skills from file
Now that you're familiar with Kernel basics, let's see how the kernel allows you to run Semantic Skills and Semantic Functions stored on disk. 

A Semantic Skill is a collection of Semantic Functions, where each function is defined with natural language that can be provided with a text file. 

Refer to our [glossary](https://github.com/microsoft/semantic-kernel/blob/main/docs/GLOSSARY.md) for an in-depth guide to the terms.

The repository includes some examples under the [samples](https://github.com/microsoft/semantic-kernel/tree/main/samples) folder.

For instance, [this](../../skills/FunSkill/Joke/skprompt.txt) is the **Joke function** part of the **FunSkill skill**:

```
WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE TOPIC BELOW.
JOKE MUST BE:
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY.
BE CREATIVE AND FUNNY. I WANT TO LAUGH.
+++++
{{$input}}
+++++
```

Note the special **`{{$input}}`** token, which is a variable that is automatically passed when invoking the function, commonly referred to as a "function parameter". 

We'll explore later how functions can accept multiple variables, as well as invoke other functions.


In the same folder you'll notice a second [config.json](../../skills/FunSkill/Joke/config.json) file. The file is optional, and is used to set some parameters for large language models like Temperature, TopP, Stop Sequences, etc.

```
{
  "schema": 1,
  "type": "completion",
  "description": "Generate a funny joke",
  "completion": {
    "max_tokens": 500,
    "temperature": 0.5,
    "top_p": 0.5
  }
}
```

Given a semantic function defined by these files, this is how to load and use a file based semantic function.

Load and configure the kernel, as usual, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [ ]:
import semantic_kernel as sk

kernel = sk.KernelBuilder.create_kernel()

useAzureOpenAI = False
model = "text-davinci-003"

# Configure AI backend used by the kernel
if useAzureOpenAI:
    api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.config.add_azure_openai_completion_backend("davinci", model, endpoint, api_key)
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.config.add_openai_completion_backend("davinci", model, api_key, org_id)

Import the skill and all its functions:

In [ ]:
# note: using skills from the skill folder
import os

skills_directory = "../../skills"
skill = kernel.import_semantic_skill_from_directory(skills_directory, "FunSkill")

How to use the skill functions, e.g. generate a joke about "*time travel to dinosaur age*":

In [ ]:
result = await kernel.run_async("time travel to dinosaur age", skill["Joke"])

print(result)

Great, now that you know how to load a skill from disk, let's show how you can [create and run a semantic function inline.](./3-semantic-function-inline.ipynb)

In [ ]:
import json
CONFIG_PATH = "../../skills/FunSkill/Joke/config.json"
PROMPT_PATH = "../../skills/FunSkill/Joke/skprompt.txt"
with open(CONFIG_PATH) as f:
    config_data = json.load(f)

with open(PROMPT_PATH) as f:
    prompt_data = f.read()

In [ ]:
from semantic_kernel.semantic_functions.prompt_template import PromptTemplate
from semantic_kernel.semantic_functions.prompt_template_config import (
    PromptTemplateConfig,
)
from semantic_kernel.semantic_functions.semantic_function_config import (
    SemanticFunctionConfig,
)

In [ ]:
config = PromptTemplateConfig()
config.schema = config_data.get("schema")
config.type = config_data.get("type")
config.description = config_data.get("description")

# Some skills may not have all completion parameters defined
config.completion = PromptTemplateConfig.CompletionConfig()
completition_dict = config_data["completion"]
config.completion.temperature = completition_dict.get("temperature")
config.completion.top_p = completition_dict.get("top_p")
config.completion.presence_penalty = completition_dict.get("presence_penalty")
config.completion.frequency_penalty = completition_dict.get("frequency_penalty")
config.completion.max_tokens = completition_dict.get("max_tokens")
config.completion.stop_sequences = completition_dict.get("stop_sequences")
config.default_backends = config_data.get("default_backends")

# Some skills may not have input parameters defined
if config_data.get("input") is not None:
    config.input = PromptTemplateConfig.InputConfig()
    config.input.parameters = []
    for parameter in config_data["input"]["parameters"]:
        config.input.parameters.append(
            PromptTemplateConfig.InputParameter(
                parameter["name"],
                parameter["description"],
                parameter["default_value"],
            )
        )

In [ ]:
# Load Prompt Template
template = PromptTemplate(prompt_data, config, kernel.prompt_template_engine)

# Prepare lambda wrapping AI logic
function_config = SemanticFunctionConfig(config, template)

In [ ]:
config

In [ ]:
function_config

In [ ]:
kernel.register_semantic_function("FunSkill", "Joke", function_config)

In [ ]:
function_config.prompt_template_config

In [ ]:
from semantic_kernel.ai.ai_exception import AIException
from semantic_kernel.ai.complete_request_settings import CompleteRequestSettings
from semantic_kernel.ai.open_ai.services.azure_text_completion import (
    AzureTextCompletion,
)
from semantic_kernel.ai.open_ai.services.open_ai_text_completion import (
    OpenAITextCompletion,
)
from semantic_kernel.configuration.backend_types import BackendType
from semantic_kernel.configuration.kernel_config import KernelConfig
from semantic_kernel.diagnostics.verify import Verify
from semantic_kernel.kernel_base import KernelBase
from semantic_kernel.kernel_exception import KernelException
from semantic_kernel.memory.semantic_text_memory_base import SemanticTextMemoryBase
from semantic_kernel.orchestration.context_variables import ContextVariables
from semantic_kernel.orchestration.sk_context import SKContext
from semantic_kernel.orchestration.sk_function import SKFunction
from semantic_kernel.orchestration.sk_function_base import SKFunctionBase
from semantic_kernel.semantic_functions.prompt_template import PromptTemplate
from semantic_kernel.semantic_functions.prompt_template_config import (
    PromptTemplateConfig,
)
from semantic_kernel.semantic_functions.semantic_function_config import (
    SemanticFunctionConfig,
)
from semantic_kernel.skill_definition.read_only_skill_collection_base import (
    ReadOnlySkillCollectionBase,
)
from semantic_kernel.skill_definition.skill_collection import SkillCollection
from semantic_kernel.skill_definition.skill_collection_base import SkillCollectionBase
from semantic_kernel.template_engine.prompt_template_engine_base import (
    PromptTemplateEngineBase,
)

In [ ]:
def temp_create_semantic_function(
    kernel, 
    skill_name: str,
    function_name: str,
    function_config: SemanticFunctionConfig,
) -> SKFunctionBase:
    function_type = function_config.prompt_template_config.type
    if not function_type == "completion":
        raise AIException(
            AIException.ErrorCodes.FunctionTypeNotSupported,
            f"Function type not supported: {function_type}",
        )

    function = SKFunction.from_semantic_config(
        skill_name, function_name, function_config
    )
    function.request_settings.update_from_completion_config(
        function_config.prompt_template_config.completion
    )

    # Connect the function to the current kernel skill
    # collection, in case the function is invoked manually
    # without a context and without a way to find other functions.
    function.set_default_skill_collection(kernel.skills)

    # TODO: allow to postpone this (use lazy init)
    # allow to create semantic functions without
    # a default backend
    backend = kernel._config.get_completion_backend(
        function_config.prompt_template_config.default_backends[0]
        if len(function_config.prompt_template_config.default_backends) > 0
        else None
    )

    function.set_ai_configuration(
        CompleteRequestSettings.from_completion_config(
            function_config.prompt_template_config.completion
        )
    )

    if backend.backend_type == BackendType.AzureOpenAI:
        Verify.not_null(
            backend.azure_open_ai, "Azure OpenAI configuration is missing"
        )
        function.set_ai_backend(
            lambda: AzureTextCompletion(
                backend.azure_open_ai.deployment_name,  # type: ignore
                backend.azure_open_ai.endpoint,  # type: ignore
                backend.azure_open_ai.api_key,  # type: ignore
                backend.azure_open_ai.api_version,  # type: ignore
                kernel._log,
            )
        )
    elif backend.backend_type == BackendType.OpenAI:
        Verify.not_null(backend.open_ai, "OpenAI configuration is missing")
        function.set_ai_backend(
            lambda: OpenAITextCompletion(
                backend.open_ai.model_id,  # type: ignore
                backend.open_ai.api_key,  # type: ignore
                backend.open_ai.org_id,  # type: ignore
                kernel._log,
            )
        )
    else:
        raise AIException(
            AIException.ErrorCodes.InvalidConfiguration,
            f"Unknown/unsupported backend type: {backend.backend_type.name}, "
            f"unable to prepare semantic function. Function description: "
            f"{function_config.prompt_template_config.description}",
        )

    return function

In [ ]:
temp_create_semantic_function(kernel, "FunSkill", "Joke", function_config)